In [1]:
import sys
import os

# change working directory
current_dir = os.getcwd() 
print(f"Current Working Directory: {current_dir}")
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
os.chdir(parent_dir)
current_dir = os.getcwd() 
print(f"Current Working Directory: {current_dir}")
#We need to be in the main directory that contains tests, models, etc folders

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-26 16:49:35.416431: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-26 16:49:35.431861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753548575.447797    2541 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753548575.452901    2541 cuda_blas.cc:1407] Unable to 

Current Working Directory: /lambda/nfs/pranjal-codebase/tests
Current Working Directory: /lambda/nfs/pranjal-codebase


In [3]:
import numpy as np
from transformers import AutoTokenizer
import torch
from coarse_grain_model import GPT2WithSlidingWindow, GemmaWithSlidingWindow

In [ ]:
##########################  Testing Gemme coarse grain masking  #########

In [6]:
"""
Tests the custom model by visualizing the attention pattern for a specific token.
"""
model_path = "./models/gemma2b-it" 
WINDOW_SIZE = 5  # Use a small window for easy verification

# 1. Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = GemmaWithSlidingWindow.from_pretrained(model_path,WINDOW_SIZE, attn_implementation="eager")
model.eval() # Set model to evaluation mode

# 2. Create sample input
text = "The brown fox wants to eat the lazy dog but there is"
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]
tokens = [tokenizer.decode(token_id) for token_id in input_ids[0]]
seq_len = len(tokens)

print(f"Input Sentence: '{text}'")
print(f"Window Size: {WINDOW_SIZE}\n")
print("-" * 50)

# 3. Perform a forward pass, requesting attention scores
with torch.no_grad():
    outputs = model(**inputs, output_attentions=True)

# `outputs.attentions` is a tuple of attention tensors, one for each layer.
# Let's inspect the first layer's attention.
# Shape: [batch_size, num_heads, query_len, key_len]
attention_layer_0 = outputs.attentions[0]

# Let's inspect the attention from the first head.
attention_head_0 = attention_layer_0[0, 0, :, :].cpu().numpy()

# 4. Pick a token to analyze and verify its attention window
token_to_check_index = 10

# Get the attention scores *from* this token *to* all other tokens
print(attention_head_0.shape)
attention_scores = attention_head_0[token_to_check_index]

# The tokens that received a non-negligible attention score
attended_indices = np.where(attention_scores > 0.00001)[0]
print("attention scores=",attention_scores)
# The actual tokens it attended to
attended_tokens = [tokens[i] for i in attended_indices]

# Calculate the expected window
expected_start_index = max(0, token_to_check_index - WINDOW_SIZE + 1)
expected_end_index = token_to_check_index
expected_window_indices = list(range(expected_start_index, expected_end_index + 1))
expected_tokens = [tokens[i] for i in expected_window_indices]

print(f"🔍 ANALYSIS FOR TOKEN '{tokens[token_to_check_index]}' (at index {token_to_check_index}):\n")

print(f"EXPECTED to attend to tokens from index {expected_start_index} to {expected_end_index}:")
print(f"==> {expected_tokens}\n")

print(f"ACTUALLY attended to tokens at indices {attended_indices.tolist()}:")
print(f"==> {attended_tokens}\n")

# 5. Assert to confirm correctness
assert sorted(attended_indices.tolist()) == sorted(expected_window_indices), \
    "Test Failed: The model did not attend to the correct sliding window!"

print("✅ TEST PASSED: The attention pattern matches the expected sliding window.")

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  3.40it/s]


Input Sentence: 'The brown fox wants to eat the lazy dog but there is'
Window Size: 5

--------------------------------------------------
(13, 13)
attention scores= [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 1.6765003e-04 6.9155376e-03 1.9207938e-02 1.5261101e-02
 9.5844775e-01 0.0000000e+00 0.0000000e+00]
🔍 ANALYSIS FOR TOKEN ' but' (at index 10):

EXPECTED to attend to tokens from index 6 to 10:
==> [' eat', ' the', ' lazy', ' dog', ' but']

ACTUALLY attended to tokens at indices [6, 7, 8, 9, 10]:
==> [' eat', ' the', ' lazy', ' dog', ' but']

✅ TEST PASSED: The attention pattern matches the expected sliding window.


In [8]:
# 2b. Generate some sample text to see the output
print("\n--- Now testing mask for Nth token prediction---")

# Use a shorter prompt for generation to see what it comes up with
generation_prompt = "brownie was then"
generation_inputs = tokenizer(generation_prompt, return_tensors="pt")

# Ensure inputs are on the same device as the model
generation_inputs = {k: v.to(model.device) for k, v in generation_inputs.items()}

with torch.no_grad():
    # Generate text using the model
    generated_output= model.generate(
        **generation_inputs,
        max_new_tokens=5,
        temperature=0.8,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        output_attentions=True,
    return_dict_in_generate=True
    )
print("-" * 50)
print("INSPECTION OF ATTENTION SCORES AFTER GENERATION")
print("-" * 50)

# `generated_output.attentions` is a tuple of tuples.
# The outer tuple iterates over each generation step.
for i, step_attentions in enumerate(generated_output.attentions):
    # `step_attentions` is a tuple for all the model layers at this step
    # We'll just inspect the first layer's attention
    layer_0_attention = step_attentions[0]
    
    # The shape is [batch_size, num_heads, query_len, key_len]
    # During generation, query_len is always 1.
    # key_len is the total sequence length so far.
    num_total_tokens = layer_0_attention.shape[-1]
    
    print(f"\n✅ Analysis for Generated Token #{i+1} (Total Sequence Length: {num_total_tokens})")
    print(f"   Attention tensor shape: {layer_0_attention.shape}")
    
    # Get scores from the first head for the new token
    scores = layer_0_attention[0, 0, -1, :].cpu().detach().numpy()
    
    print(f"   Scores from new token to all {num_total_tokens} previous tokens:")
    np.set_printoptions(precision=3, suppress=True)
    print(f"   {scores}")
    
    # Check if the scores respect the window size
    num_non_window_tokens = max(0, num_total_tokens - WINDOW_SIZE)
    first_few_scores_sum = np.sum(scores[:num_non_window_tokens])
    
    if first_few_scores_sum > 0.01: # Check if there's significant attention outside the window
         print(f"   🚨 DIAGNOSIS: FAILED. Attention was paid to tokens outside the expected window of size {WINDOW_SIZE}.")
    else:
         print(f"   ✅ DIAGNOSIS: PASSED. Attention correctly constrained.")

print("\n" + "-" * 50)
print("Generation Complete.")
decoded_text = tokenizer.decode(generated_output.sequences[0])
print(f"Final Output: '{decoded_text}'")


--- Now testing mask for Nth token prediction---
--------------------------------------------------
INSPECTION OF ATTENTION SCORES AFTER GENERATION
--------------------------------------------------

✅ Analysis for Generated Token #1 (Total Sequence Length: 5)
   Attention tensor shape: torch.Size([1, 8, 5, 5])
   Scores from new token to all 5 previous tokens:
   [0.173 0.    0.013 0.093 0.72 ]
   ✅ DIAGNOSIS: PASSED. Attention correctly constrained.

✅ Analysis for Generated Token #2 (Total Sequence Length: 6)
   Attention tensor shape: torch.Size([1, 8, 1, 6])
   Scores from new token to all 6 previous tokens:
   [0.    0.    0.005 0.868 0.08  0.047]
   ✅ DIAGNOSIS: PASSED. Attention correctly constrained.

✅ Analysis for Generated Token #3 (Total Sequence Length: 7)
   Attention tensor shape: torch.Size([1, 8, 1, 7])
   Scores from new token to all 7 previous tokens:
   [0.    0.    0.006 0.086 0.171 0.451 0.286]
   ✅ DIAGNOSIS: PASSED. Attention correctly constrained.

✅ Analysis

In [2]:
######################## Testing GPT2 coarse grain masking ######################

In [2]:
"""
Tests the custom model by visualizing the attention pattern for a specific token.
"""
model_path = "./models/gpt2" 
WINDOW_SIZE = 5  # Use a small window for easy verification

# 1. Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2WithSlidingWindow.from_pretrained(model_path,WINDOW_SIZE)
model.eval() # Set model to evaluation mode

# 2. Create sample input
text = "The brown fox wants to eat the lazy dog but there is"
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]
tokens = [tokenizer.decode(token_id) for token_id in input_ids[0]]
seq_len = len(tokens)

print(f"Input Sentence: '{text}'")
print(f"Window Size: {WINDOW_SIZE}\n")
print("-" * 50)

# 3. Perform a forward pass, requesting attention scores
with torch.no_grad():
    outputs = model(**inputs, output_attentions=True)

# `outputs.attentions` is a tuple of attention tensors, one for each layer.
# Let's inspect the first layer's attention.
# Shape: [batch_size, num_heads, query_len, key_len]
attention_layer_0 = outputs.attentions[0]

# Let's inspect the attention from the first head.
attention_head_0 = attention_layer_0[0, 0, :, :].cpu().numpy()

# 4. Pick a token to analyze and verify its attention window
token_to_check_index = 10

# Get the attention scores *from* this token *to* all other tokens
print(attention_head_0.shape)
attention_scores = attention_head_0[token_to_check_index]

# The tokens that received a non-negligible attention score
attended_indices = np.where(attention_scores > 0.001)[0]
print("attention scores=",attention_scores)
# The actual tokens it attended to
attended_tokens = [tokens[i] for i in attended_indices]

# Calculate the expected window
expected_start_index = max(0, token_to_check_index - WINDOW_SIZE + 1)
expected_end_index = token_to_check_index
expected_window_indices = list(range(expected_start_index, expected_end_index + 1))
expected_tokens = [tokens[i] for i in expected_window_indices]

print(f"🔍 ANALYSIS FOR TOKEN '{tokens[token_to_check_index]}' (at index {token_to_check_index}):\n")

print(f"EXPECTED to attend to tokens from index {expected_start_index} to {expected_end_index}:")
print(f"==> {expected_tokens}\n")

print(f"ACTUALLY attended to tokens at indices {attended_indices.tolist()}:")
print(f"==> {attended_tokens}\n")

# 5. Assert to confirm correctness
assert sorted(attended_indices.tolist()) == sorted(expected_window_indices), \
    "Test Failed: The model did not attend to the correct sliding window!"

print("✅ TEST PASSED: The attention pattern matches the expected sliding window.")

`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Input Sentence: 'The brown fox wants to eat the lazy dog but there is'
Window Size: 5

--------------------------------------------------
(12, 12)
attention scores= [0.         0.         0.         0.         0.         0.
 0.06263476 0.47507763 0.21666087 0.16513513 0.08049163 0.        ]
🔍 ANALYSIS FOR TOKEN ' there' (at index 10):

EXPECTED to attend to tokens from index 6 to 10:
==> [' the', ' lazy', ' dog', ' but', ' there']

ACTUALLY attended to tokens at indices [6, 7, 8, 9, 10]:
==> [' the', ' lazy', ' dog', ' but', ' there']

✅ TEST PASSED: The attention pattern matches the expected sliding window.


In [4]:
# 2b. Generate some sample text to see the output
print("\n--- Generating Sample Text ---")

# Use a shorter prompt for generation to see what it comes up with
generation_prompt = "brownie was then"
generation_inputs = tokenizer(generation_prompt, return_tensors="pt")

# Ensure inputs are on the same device as the model
generation_inputs = {k: v.to(model.device) for k, v in generation_inputs.items()}

with torch.no_grad():
    # Generate text using the model
    generated_output= model.generate(
        **generation_inputs,
        max_new_tokens=3,
        temperature=0.8,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        output_attentions=True,
    return_dict_in_generate=True
    )
print("-" * 50)
print("INSPECTION OF ATTENTION SCORES AFTER GENERATION")
print("-" * 50)

# `generated_output.attentions` is a tuple of tuples.
# The outer tuple iterates over each generation step.
for i, step_attentions in enumerate(generated_output.attentions):
    # `step_attentions` is a tuple for all the model layers at this step
    # We'll just inspect the first layer's attention
    layer_0_attention = step_attentions[0]
    
    # The shape is [batch_size, num_heads, query_len, key_len]
    # During generation, query_len is always 1.
    # key_len is the total sequence length so far.
    num_total_tokens = layer_0_attention.shape[-1]
    
    print(f"\n✅ Analysis for Generated Token #{i+1} (Total Sequence Length: {num_total_tokens})")
    print(f"   Attention tensor shape: {layer_0_attention.shape}")
    
    # Get scores from the first head for the new token
    scores = layer_0_attention[0, 0, -1, :].cpu().detach().numpy()
    
    print(f"   Scores from new token to all {num_total_tokens} previous tokens:")
    np.set_printoptions(precision=3, suppress=True)
    print(f"   {scores}")
    
    # Check if the scores respect the window size
    num_non_window_tokens = max(0, num_total_tokens - WINDOW_SIZE)
    first_few_scores_sum = np.sum(scores[:num_non_window_tokens])
    
    if first_few_scores_sum > 0.01: # Check if there's significant attention outside the window
         print(f"   🚨 DIAGNOSIS: FAILED. Attention was paid to tokens outside the expected window of size {WINDOW_SIZE}.")
    else:
         print(f"   ✅ DIAGNOSIS: PASSED. Attention correctly constrained.")

print("\n" + "-" * 50)
print("Generation Complete.")
decoded_text = tokenizer.decode(generated_output.sequences[0])
print(f"Final Output: '{decoded_text}'")


--- Generating Sample Text ---
--------------------------------------------------
INSPECTION OF ATTENTION SCORES AFTER GENERATION
--------------------------------------------------

✅ Analysis for Generated Token #1 (Total Sequence Length: 4)
   Attention tensor shape: torch.Size([1, 12, 4, 4])
   Scores from new token to all 4 previous tokens:
   [0.465 0.083 0.351 0.102]
   ✅ DIAGNOSIS: PASSED. Attention correctly constrained.

✅ Analysis for Generated Token #2 (Total Sequence Length: 5)
   Attention tensor shape: torch.Size([1, 12, 1, 5])
   Scores from new token to all 5 previous tokens:
   [0.403 0.163 0.223 0.052 0.159]
   ✅ DIAGNOSIS: PASSED. Attention correctly constrained.

✅ Analysis for Generated Token #3 (Total Sequence Length: 6)
   Attention tensor shape: torch.Size([1, 12, 1, 6])
   Scores from new token to all 6 previous tokens:
   [0.    0.188 0.304 0.213 0.234 0.061]
   ✅ DIAGNOSIS: PASSED. Attention correctly constrained.

-------------------------------------------

In [1]:
#########Testing Anindita's Coarse grain function##################


In [5]:
# ==============================
# SECTION 2 — Coarse-Graining
# ==============================

def coarsegrain_attention(model, span):
    """
    Monkey-patches the GPT2 attention blocks to enforce a banded diagonal attention matrix:
    - Attention is restricted to keys within ±span//2 of each query position.
    - Within the span, standard softmax attention is applied.
    - Outside the span, outputs of attention factor are zero.
    """

    half_span = span // 2

    def custom_attn_scores(self, query, key, value, attention_mask=None, head_mask=None):
        scores = torch.matmul(query, key.transpose(-1, -2)) / (value.size(-1) ** 0.5)
        q_len, k_len = scores.size(-2), scores.size(-1)
        assert q_len == k_len #attention masking assumes square attention blocks (query and key lengths match).

        # sets up the local attention block
        span_mask = torch.zeros((q_len, k_len), dtype=torch.bool, device=scores.device)
        for i in range(q_len):
            left = max(0, i - half_span)
            right = min(k_len, i + half_span + 1)
            span_mask[i, left:right] = 1


        masked_scores = scores.masked_fill(~span_mask, float('-inf'))
        attention_weights = torch.nn.functional.softmax(masked_scores, dim=-1)
        outputs = torch.matmul(attention_weights, value)

        # Set outputs outside the local attention block to zero, in case some elements of original attention span are 10e-30
        diag_mask = span_mask.diagonal().unsqueeze(-1).expand(-1, value.size(-1))
        outputs = outputs * diag_mask

        return outputs, None

    for block in model.transformer.h:
        block.attn._attn = custom_attn_scores.__get__(block.attn)

In [17]:
# Example usage:
WINDOW_SIZE = 6  # Use a small window for easy verification
model2 = GPT2LMHeadModel.from_pretrained("./models/gpt2")
coarsegrain_attention(model2, span=WINDOW_SIZE)

model2.eval() # Set model to evaluation mode

# 2. Create sample input
text = "The brown fox wants to eat the lazy dog but there is"
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]
tokens = [tokenizer.decode(token_id) for token_id in input_ids[0]]
seq_len = len(tokens)

print(f"Input Sentence: '{text}'")
print(f"Window Size: {WINDOW_SIZE}\n")
print("-" * 50)

# 3. Perform a forward pass, requesting attention scores
with torch.no_grad():
    outputs = model2(**inputs, output_attentions=True)

# `outputs.attentions` is a tuple of attention tensors, one for each layer.
# Let's inspect the first layer's attention.
# Shape: [batch_size, num_heads, query_len, key_len]
attention_layer_0 = outputs.attentions[0]

# Let's inspect the attention from the first head.
attention_head_0 = attention_layer_0[0, 0, :, :].cpu().numpy()

# 4. Pick a token to analyze and verify its attention window
token_to_check_index = 7

half_span = WINDOW_SIZE // 2 # Use the same logic as the function

# Get the attention scores *from* this token *to* all other tokens
print(attention_head_0.shape)
attention_scores = attention_head_0[token_to_check_index]

# The tokens that received a non-negligible attention score
attended_indices = np.where(attention_scores > 0.001)[0]

# The actual tokens it attended to
attended_tokens = [tokens[i] for i in attended_indices]

# Calculate the expected window
expected_start_index = max(0, token_to_check_index - half_span)
expected_end_index = min(seq_len - 1, token_to_check_index + half_span)
expected_window_indices = list(range(expected_start_index, expected_end_index + 1))
expected_tokens = [tokens[i] for i in expected_window_indices]

print(f"🔍 ANALYSIS FOR TOKEN '{tokens[token_to_check_index]}' (at index {token_to_check_index}):\n")

print(f"EXPECTED to attend to tokens from index {expected_start_index} to {expected_end_index}:")
print(f"==> {expected_tokens}\n")

print(f"ACTUALLY attended to tokens at indices {attended_indices.tolist()}:")
print(f"==> {attended_tokens}\n")

# 5. Assert to confirm correctness
assert sorted(attended_indices.tolist()) == sorted(expected_window_indices), \
    "Test Failed: The model did not attend to the correct sliding window!"

print("✅ TEST PASSED: The attention pattern matches the expected sliding window.")

Input Sentence: 'The brown fox wants to eat the lazy dog but there is'
Window Size: 6

--------------------------------------------------
(12, 12)
🔍 ANALYSIS FOR TOKEN ' lazy' (at index 7):

EXPECTED to attend to tokens from index 4 to 10:
==> [' to', ' eat', ' the', ' lazy', ' dog', ' but', ' there']

ACTUALLY attended to tokens at indices [0, 1, 2, 3, 4, 5, 6, 7]:
==> ['The', ' brown', ' fox', ' wants', ' to', ' eat', ' the', ' lazy']



AssertionError: Test Failed: The model did not attend to the correct sliding window!

In [14]:
# 2b. Generate some sample text to see the output
print("\n--- Generating Sample Text ---")

# Use a shorter prompt for generation to see what it comes up with
generation_prompt = "The brownie was then"
generation_inputs = tokenizer(generation_prompt, return_tensors="pt")

# Ensure inputs are on the same device as the model
generation_inputs = {k: v.to(model2.device) for k, v in generation_inputs.items()}

with torch.no_grad():
    # Generate text using the model
    generated_output= model2.generate(
        **generation_inputs,
        max_new_tokens=3,
        temperature=0.8,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        output_attentions=True,
    return_dict_in_generate=True
    )
print("-" * 50)
print("INSPECTION OF ATTENTION SCORES AFTER GENERATION")
print("-" * 50)

# `generated_output.attentions` is a tuple of tuples.
# The outer tuple iterates over each generation step.
for i, step_attentions in enumerate(generated_output.attentions):
    # `step_attentions` is a tuple for all the model layers at this step
    # We'll just inspect the first layer's attention
    layer_0_attention = step_attentions[0]
    
    # The shape is [batch_size, num_heads, query_len, key_len]
    # During generation, query_len is always 1.
    # key_len is the total sequence length so far.
    num_total_tokens = layer_0_attention.shape[-1]
    
    print(f"\n✅ Analysis for Generated Token #{i+1} (Total Sequence Length: {num_total_tokens})")
    print(f"   Attention tensor shape: {layer_0_attention.shape}")
    
    # Get scores from the first head for the new token
    scores = layer_0_attention[0, 0, -1, :].cpu().detach().numpy()
    
    print(f"   Scores from new token to all {num_total_tokens} previous tokens:")
    np.set_printoptions(precision=3, suppress=True)
    print(f"   {scores}")
    
    # Check if the scores respect the window size
    num_non_window_tokens = max(0, num_total_tokens - WINDOW_SIZE)
    first_few_scores_sum = np.sum(scores[:num_non_window_tokens])
    
    if first_few_scores_sum > 0.01: # Check if there's significant attention outside the window
         print(f"   🚨 DIAGNOSIS: FAILED. Attention was paid to tokens outside the expected window of size {WINDOW_SIZE}.")
    else:
         print(f"   ✅ DIAGNOSIS: PASSED. Attention correctly constrained.")

print("\n" + "-" * 50)
print("Generation Complete.")
decoded_text = tokenizer.decode(generated_output.sequences[0])
print(f"Final Output: '{decoded_text}'")


--- Generating Sample Text ---
--------------------------------------------------
INSPECTION OF ATTENTION SCORES AFTER GENERATION
--------------------------------------------------

✅ Analysis for Generated Token #1 (Total Sequence Length: 5)
   Attention tensor shape: torch.Size([1, 12, 5, 5])
   Scores from new token to all 5 previous tokens:
   [0.482 0.093 0.059 0.281 0.085]
   ✅ DIAGNOSIS: PASSED. Attention correctly constrained.

✅ Analysis for Generated Token #2 (Total Sequence Length: 6)
   Attention tensor shape: torch.Size([1, 12, 1, 6])
   Scores from new token to all 6 previous tokens:
   [0.408 0.135 0.124 0.129 0.123 0.08 ]
   🚨 DIAGNOSIS: FAILED. Attention was paid to tokens outside the expected window of size 5.

✅ Analysis for Generated Token #3 (Total Sequence Length: 7)
   Attention tensor shape: torch.Size([1, 12, 1, 7])
   Scores from new token to all 7 previous tokens:
   [0.332 0.241 0.076 0.136 0.103 0.083 0.028]
   🚨 DIAGNOSIS: FAILED. Attention was paid to to